In [7]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import numpy as np

%run db.py
%run database.py
%run utils.py

'2023-05-01'

In [ ]:
cgreen = '#27ae60'
cblue = '#2980b9'
cred = '#c0392b'
corange = '#d35400'
cgrey = '#7f8c8d'
cdark = '#2c3e50'
cwithe = '#ecf0f1'

figsize = (10, 6)

In [ ]:
spec = {
    'data_base': 'Mês',
    'anterior': dt.year - 1,
    'meta': 'Meta',
    'arrecadado': dt.year,
    'dif_anterior': f'{dt.year} - {dt.year - 1}',
    'dif_perc_ant': f'{dt.year} / {dt.year - 1}',
    'dif_meta': f'{dt.year} - Meta',
    'dif_perc_meta': f'{dt.year} / Meta'
}
fmt = {
    'data_base': date_to_month_name,
    'anterior': money_formatter,
    'meta': money_formatter,
    'arrecadado': money_formatter,
    'dif_anterior': money_formatter,
    'dif_perc_ant': perc0_formatter,
    'dif_meta': money_formatter,
    'dif_perc_meta': perc0_formatter,
}

In [ ]:
def plot_mensal(df, titulo):
    fig, ax = plt.subplots(figsize=figsize)
    fig.suptitle(titulo, x=0, fontsize=16)
    plt.title('valores mensais', x=0)
    x = df['data_base'].dt.strftime('%b')
    bar_width = 0.25
    x_pos = np.arange(len(x))
    plt.bar(x_pos - bar_width, df['anterior'], color=cblue, label=str(dt.year - 1), width=bar_width, align='center')
    plt.bar(x_pos, df['meta'], color=cgrey, label='Meta', width=bar_width, align='center')
    plt.bar(x_pos + bar_width, df['arrecadado'], color=cgreen, label='Arrecadado', width=bar_width, align='center')
    plt.ylim(bottom=0)
    ax.yaxis.set_major_formatter(int_formatter)
    ax.set_xticks(x_pos)
    ax.set_xticklabels(x)
    plt.legend(loc='upper left', ncol=3)
    plt.show()
    

def plot_acumulado(df, titulo):
    fig, ax = plt.subplots(figsize=figsize)
    fig.suptitle(titulo, x=0, fontsize=16)
    plt.title('valores acumulados', x=0)
    x = df['data_base'].dt.strftime('%b')
    plt.plot(x, df['anterior'], color=cblue, label=str(dt.year - 1), marker='o')
    plt.plot(x, df['meta'], color=cgrey, label='Meta', linestyle='--')
    plt.plot(x, df['arrecadado'], color=cgreen, label='Arrecadado', marker='^')
    plt.ylim(bottom=0)
    ax.yaxis.set_major_formatter(int_formatter)
    plt.legend(loc='upper left', ncol=3)
    plt.show()

    
def prepara_df_acumulado(df):
    df['arrecadado'] = df['arrecadado'].cumsum()
    df['meta'] = df['meta'].cumsum()
    df['anterior'] = df['anterior'].cumsum()
    df['dif_meta'] = df['arrecadado'] - df['meta']
    df['dif_perc_meta'] = (df['arrecadado'] / df['meta']) - 1
    df['dif_anterior'] = df['arrecadado'] - df['anterior']
    df['dif_perc_ant'] = (df['arrecadado'] / df['anterior']) - 1
    return df

def prepara_df_mensal(df):
    df['dif_meta'] = df['arrecadado'] - df['meta']
    df['dif_perc_meta'] = (df['arrecadado'] / df['meta']) - 1
    df['dif_anterior'] = df['arrecadado'] - df['anterior']
    df['dif_perc_ant'] = (df['arrecadado'] / df['anterior']) - 1
    return df


# Receitas

Nesta parte são demonstradas as receitas da **Prefeitura**, excluídas, portanto, as receias do *RPPS*. Também são excluídas as receitas *intra-orçamentárias*.

As receitas são apresentadas em seus valores líquidos das deduções da receita.

## Receita Total

In [ ]:
nome_receita = 'Receita Total'
sql = f'''
SELECT
    data_base,
    SUM(receita_realizada) AS arrecadado,
    SUM(meta_mensal) AS meta
FROM pad."RECEITA"

WHERE
    controle = {controle_mes}
    AND entidade LIKE 'pm'
    AND classe_receita NOT LIKE 'intra'
GROUP BY
    data_base
ORDER BY
    data_base ASC
'''
#print(sql)
receita_atual = sqldf(sql)
#receita_atual

sql = f'''
SELECT
    data_base,
    SUM(receita_realizada) AS arrecadado
FROM pad."RECEITA"

WHERE
    controle = {controle_mes_ano_anterior}
    AND entidade LIKE 'pm'
    AND classe_receita NOT LIKE 'intra'
GROUP BY
    data_base
ORDER BY
    data_base ASC
'''
#print(sql)
receita_anterior = sqldf(sql)
#receita_anterior

receita = receita_atual
receita['anterior'] = receita_anterior['arrecadado']
receita['data_base'] = pd.to_datetime(receita['data_base'], format='%Y-%m-%d')
#receita.dtypes

In [ ]:
acumulado = prepara_df_acumulado(receita.copy())
plot_acumulado(acumulado, nome_receita)

In [ ]:
df = df_table(acumulado.copy(), spec, fmt)
df.style.set_caption(f'{nome_receita} - valores acumulados - {dt.year}').hide(axis='index')
#df

In [ ]:
mensal = prepara_df_mensal(receita.copy())
plot_mensal(mensal, nome_receita)

In [ ]:
df = df_table(mensal.copy(), spec, fmt)
df.style.set_caption(f'{nome_receita} - valores mensais - {dt.year}').hide(axis='index')

## Receita Corrente

In [ ]:
nome_receita = 'Receita Corrente'
sql = f'''
SELECT
    data_base,
    SUM(receita_realizada) AS arrecadado,
    SUM(meta_mensal) AS meta
FROM pad."RECEITA"

WHERE
    controle = {controle_mes}
    AND entidade LIKE 'pm'
    AND classe_receita NOT LIKE 'intra'
    AND receita_base LIKE '1%%'
GROUP BY
    data_base
ORDER BY
    data_base ASC
'''
#print(sql)
receita_atual = sqldf(sql)
#receita_atual

sql = f'''
SELECT
    data_base,
    SUM(receita_realizada) AS arrecadado
FROM pad."RECEITA"

WHERE
    controle = {controle_mes_ano_anterior}
    AND entidade LIKE 'pm'
    AND classe_receita NOT LIKE 'intra'
    AND receita_base LIKE '1%%'
GROUP BY
    data_base
ORDER BY
    data_base ASC
'''
#print(sql)
receita_anterior = sqldf(sql)
#receita_anterior

receita = receita_atual
receita['anterior'] = receita_anterior['arrecadado']
receita['data_base'] = pd.to_datetime(receita['data_base'], format='%Y-%m-%d')
#receita.dtypes

In [ ]:
acumulado = prepara_df_acumulado(receita.copy())
plot_acumulado(acumulado, nome_receita)

In [ ]:
df = df_table(acumulado.copy(), spec, fmt)
df.style.set_caption(f'{nome_receita} - valores acumulados - {dt.year}').hide(axis='index')
#df

In [ ]:
mensal = prepara_df_mensal(receita.copy())
plot_mensal(mensal, nome_receita)

In [ ]:
df = df_table(mensal.copy(), spec, fmt)
df.style.set_caption(f'{nome_receita} - valores mensais - {dt.year}').hide(axis='index')

## Arrecadação Própria

As receitas de **arrecadação prórpria** são as receitas correntes exceto as transferências e as receitas de rendimenos financeiros e expressam aquelas receitas cujo esforço de arrecadação é exclusivo do Município.yt9923928i

In [ ]:
nome_receita = 'Arrecadação Própria'
sql = f'''
SELECT
    data_base,
    SUM(receita_realizada) AS arrecadado,
    SUM(meta_mensal) AS meta
FROM pad."RECEITA"

WHERE
    controle = {controle_mes}
    AND entidade LIKE 'pm'
    AND classe_receita NOT LIKE 'intra'
    AND receita_base NOT LIKE '17%%'
    AND receita_base NOT LIKE '132%%'
GROUP BY
    data_base
ORDER BY
    data_base ASC
'''
#print(sql)
receita_atual = sqldf(sql)
#receita_atual

sql = f'''
SELECT
    data_base,
    SUM(receita_realizada) AS arrecadado
FROM pad."RECEITA"

WHERE
    controle = {controle_mes_ano_anterior}
    AND entidade LIKE 'pm'
    AND classe_receita NOT LIKE 'intra'
    AND receita_base LIKE '1%%'
    AND receita_base NOT LIKE '17%%'
    AND receita_base NOT LIKE '132%%'
GROUP BY
    data_base
ORDER BY
    data_base ASC
'''
#print(sql)
receita_anterior = sqldf(sql)
#receita_anterior

receita = receita_atual
receita['anterior'] = receita_anterior['arrecadado']
receita['data_base'] = pd.to_datetime(receita['data_base'], format='%Y-%m-%d')
#receita.dtypes

In [ ]:
acumulado = prepara_df_acumulado(receita.copy())
plot_acumulado(acumulado, nome_receita)

In [ ]:
df = df_table(acumulado.copy(), spec, fmt)
df.style.set_caption(f'{nome_receita} - valores acumulados - {dt.year}').hide(axis='index')
#df

In [ ]:
mensal = prepara_df_mensal(receita.copy())
plot_mensal(mensal, nome_receita)

In [ ]:
df = df_table(mensal.copy(), spec, fmt)
df.style.set_caption(f'{nome_receita} - valores mensais - {dt.year}').hide(axis='index')

### Receita da Dívida Ativa

In [ ]:
nome_receita = 'Receita da Dívida Ativa'
sql = f'''
SELECT
    data_base,
    SUM(receita_realizada) AS arrecadado,
    SUM(meta_mensal) AS meta
FROM pad."RECEITA"

WHERE
    controle = {controle_mes}
    AND entidade LIKE 'pm'
    AND classe_receita NOT LIKE 'intra'
    AND tipo_receita IN (3, 4, 5, 6, 7, 8)
GROUP BY
    data_base
ORDER BY
    data_base ASC
'''
#print(sql)
receita_atual = sqldf(sql)
#receita_atual

sql = f'''
SELECT
    data_base,
    SUM(receita_realizada) AS arrecadado
FROM pad."RECEITA"

WHERE
    controle = {controle_mes_ano_anterior}
    AND entidade LIKE 'pm'
    AND classe_receita NOT LIKE 'intra'
    AND tipo_receita IN (3, 4, 5, 6, 7, 8)
GROUP BY
    data_base
ORDER BY
    data_base ASC
'''
#print(sql)
receita_anterior = sqldf(sql)
#receita_anterior

receita = receita_atual
receita['anterior'] = receita_anterior['arrecadado']
receita['data_base'] = pd.to_datetime(receita['data_base'], format='%Y-%m-%d')
#receita.dtypes

In [ ]:
acumulado = prepara_df_acumulado(receita.copy())
plot_acumulado(acumulado, nome_receita)

In [ ]:
df = df_table(acumulado.copy(), spec, fmt)
df.style.set_caption(f'{nome_receita} - valores acumulados - {dt.year}').hide(axis='index')
#df

In [ ]:
mensal = prepara_df_mensal(receita.copy())
plot_mensal(mensal, nome_receita)

In [ ]:
df = df_table(mensal.copy(), spec, fmt)
df.style.set_caption(f'{nome_receita} - valores mensais - {dt.year}').hide(axis='index')

## Transferências Correntes

In [ ]:
nome_receita = 'Transferências Correntes'
sql = f'''
SELECT
    data_base,
    SUM(receita_realizada) AS arrecadado,
    SUM(meta_mensal) AS meta
FROM pad."RECEITA"

WHERE
    controle = {controle_mes}
    AND entidade LIKE 'pm'
    AND classe_receita NOT LIKE 'intra'
    AND receita_base LIKE '17%%'
GROUP BY
    data_base
ORDER BY
    data_base ASC
'''
#print(sql)
receita_atual = sqldf(sql)
#receita_atual

sql = f'''
SELECT
    data_base,
    SUM(receita_realizada) AS arrecadado
FROM pad."RECEITA"

WHERE
    controle = {controle_mes_ano_anterior}
    AND entidade LIKE 'pm'
    AND classe_receita NOT LIKE 'intra'
    AND receita_base LIKE '17%%'
GROUP BY
    data_base
ORDER BY
    data_base ASC
'''
#print(sql)
receita_anterior = sqldf(sql)
#receita_anterior

receita = receita_atual
receita['anterior'] = receita_anterior['arrecadado']
receita['data_base'] = pd.to_datetime(receita['data_base'], format='%Y-%m-%d')
#receita.dtypes

In [ ]:
acumulado = prepara_df_acumulado(receita.copy())
plot_acumulado(acumulado, nome_receita)

In [ ]:
df = df_table(acumulado.copy(), spec, fmt)
df.style.set_caption(f'{nome_receita} - valores acumulados - {dt.year}').hide(axis='index')
#df

In [ ]:
mensal = prepara_df_mensal(receita.copy())
plot_mensal(mensal, nome_receita)

In [ ]:
df = df_table(mensal.copy(), spec, fmt)
df.style.set_caption(f'{nome_receita} - valores mensais - {dt.year}').hide(axis='index')

### Transferências do Fundeb

Engloba os valores recebidos do Fundeb. Não são considerados os rendimentos financeiros nem os valores deduzidos para o Fundeb.

In [ ]:
nome_receita = 'Transferências do Fundeb'
sql = f'''
SELECT
    data_base,
    SUM(receita_realizada) AS arrecadado,
    SUM(meta_mensal) AS meta
FROM pad."RECEITA"

WHERE
    controle = {controle_mes}
    AND entidade LIKE 'pm'
    AND classe_receita NOT LIKE 'intra'
    AND receita_base LIKE '17%%'
    AND fonte_recurso IN (540, 541, 542, 543)
GROUP BY
    data_base
ORDER BY
    data_base ASC
'''
#print(sql)
receita_atual = sqldf(sql)
#receita_atual

sql = f'''
SELECT
    data_base,
    SUM(receita_realizada) AS arrecadado
FROM pad."RECEITA"

WHERE
    controle = {controle_mes_ano_anterior}
    AND entidade LIKE 'pm'
    AND classe_receita NOT LIKE 'intra'
    AND receita_base LIKE '17%%'
    AND fonte_recurso IN (540, 541, 542, 543)
GROUP BY
    data_base
ORDER BY
    data_base ASC
'''
#print(sql)
receita_anterior = sqldf(sql)
#receita_anterior

receita = receita_atual
receita['anterior'] = receita_anterior['arrecadado']
receita['data_base'] = pd.to_datetime(receita['data_base'], format='%Y-%m-%d')
#receita.dtypes

In [ ]:
acumulado = prepara_df_acumulado(receita.copy())
plot_acumulado(acumulado, nome_receita)

In [ ]:
df = df_table(acumulado.copy(), spec, fmt)
df.style.set_caption(f'{nome_receita} - valores acumulados - {dt.year}').hide(axis='index')
#df

In [ ]:
mensal = prepara_df_mensal(receita.copy())
plot_mensal(mensal, nome_receita)

In [ ]:
df = df_table(mensal.copy(), spec, fmt)
df.style.set_caption(f'{nome_receita} - valores mensais - {dt.year}').hide(axis='index')

### Transferências recebidas da União

Engloba as transferências correntes recebidas da União, inclusive convênios.

In [ ]:
nome_receita = 'Transferências recebidas da União'
sql = f'''
SELECT
    data_base,
    SUM(receita_realizada) AS arrecadado,
    SUM(meta_mensal) AS meta
FROM pad."RECEITA"

WHERE
    controle = {controle_mes}
    AND entidade LIKE 'pm'
    AND classe_receita NOT LIKE 'intra'
    AND receita_base LIKE '171%%'
GROUP BY
    data_base
ORDER BY
    data_base ASC
'''
#print(sql)
receita_atual = sqldf(sql)
#receita_atual

sql = f'''
SELECT
    data_base,
    SUM(receita_realizada) AS arrecadado
FROM pad."RECEITA"

WHERE
    controle = {controle_mes_ano_anterior}
    AND entidade LIKE 'pm'
    AND classe_receita NOT LIKE 'intra'
    AND receita_base LIKE '171%%'
GROUP BY
    data_base
ORDER BY
    data_base ASC
'''
#print(sql)
receita_anterior = sqldf(sql)
#receita_anterior

receita = receita_atual
receita['anterior'] = receita_anterior['arrecadado']
receita['data_base'] = pd.to_datetime(receita['data_base'], format='%Y-%m-%d')
#receita.dtypes

In [ ]:
acumulado = prepara_df_acumulado(receita.copy())
plot_acumulado(acumulado, nome_receita)

In [ ]:
df = df_table(acumulado.copy(), spec, fmt)
df.style.set_caption(f'{nome_receita} - valores acumulados - {dt.year}').hide(axis='index')
#df

In [ ]:
mensal = prepara_df_mensal(receita.copy())
plot_mensal(mensal, nome_receita)

In [ ]:
df = df_table(mensal.copy(), spec, fmt)
df.style.set_caption(f'{nome_receita} - valores mensais - {dt.year}').hide(axis='index')

#### FPM

Engloba as cotas mensais e extras.

In [ ]:
nome_receita = 'FPM'
sql = f'''
SELECT
    data_base,
    SUM(receita_realizada) AS arrecadado,
    SUM(meta_mensal) AS meta
FROM pad."RECEITA"

WHERE
    controle = {controle_mes}
    AND entidade LIKE 'pm'
    AND classe_receita NOT LIKE 'intra'
    AND receita_base LIKE '171151%%'
GROUP BY
    data_base
ORDER BY
    data_base ASC
'''
#print(sql)
receita_atual = sqldf(sql)
#receita_atual

sql = f'''
SELECT
    data_base,
    SUM(receita_realizada) AS arrecadado
FROM pad."RECEITA"

WHERE
    controle = {controle_mes_ano_anterior}
    AND entidade LIKE 'pm'
    AND classe_receita NOT LIKE 'intra'
    AND receita_base LIKE '1718012%%'
    OR receita_base LIKE '1718013%%'
    OR receita_base LIKE '1718013%%'
GROUP BY
    data_base
ORDER BY
    data_base ASC
'''
#print(sql)
receita_anterior = sqldf(sql)
#receita_anterior

receita = receita_atual
receita['anterior'] = receita_anterior['arrecadado']
receita['data_base'] = pd.to_datetime(receita['data_base'], format='%Y-%m-%d')
#receita.dtypes

In [ ]:
acumulado = prepara_df_acumulado(receita.copy())
plot_acumulado(acumulado, nome_receita)

In [ ]:
df = df_table(acumulado.copy(), spec, fmt)
df.style.set_caption(f'{nome_receita} - valores acumulados - {dt.year}').hide(axis='index')
#df

In [ ]:
mensal = prepara_df_mensal(receita.copy())
plot_mensal(mensal, nome_receita)

In [ ]:
df = df_table(mensal.copy(), spec, fmt)
df.style.set_caption(f'{nome_receita} - valores mensais - {dt.year}').hide(axis='index')

### Transferências recebidas do Estado

Engloaba as transferências correntes recebidas do Estado, inclusive convênios.

In [ ]:
nome_receita = 'Transferências recebidas do Estado'
sql = f'''
SELECT
    data_base,
    SUM(receita_realizada) AS arrecadado,
    SUM(meta_mensal) AS meta
FROM pad."RECEITA"

WHERE
    controle = {controle_mes}
    AND entidade LIKE 'pm'
    AND classe_receita NOT LIKE 'intra'
    AND receita_base LIKE '172%%'
GROUP BY
    data_base
ORDER BY
    data_base ASC
'''
#print(sql)
receita_atual = sqldf(sql)
#receita_atual

sql = f'''
SELECT
    data_base,
    SUM(receita_realizada) AS arrecadado
FROM pad."RECEITA"

WHERE
    controle = {controle_mes_ano_anterior}
    AND entidade LIKE 'pm'
    AND classe_receita NOT LIKE 'intra'
    AND receita_base LIKE '172%%'
GROUP BY
    data_base
ORDER BY
    data_base ASC
'''
#print(sql)
receita_anterior = sqldf(sql)
#receita_anterior

receita = receita_atual
receita['anterior'] = receita_anterior['arrecadado']
receita['data_base'] = pd.to_datetime(receita['data_base'], format='%Y-%m-%d')
#receita.dtypes

In [ ]:
acumulado = prepara_df_acumulado(receita.copy())
plot_acumulado(acumulado, nome_receita)

In [ ]:
df = df_table(acumulado.copy(), spec, fmt)
df.style.set_caption(f'{nome_receita} - valores acumulados - {dt.year}').hide(axis='index')
#df

In [ ]:
mensal = prepara_df_mensal(receita.copy())
plot_mensal(mensal, nome_receita)

In [ ]:
df = df_table(mensal.copy(), spec, fmt)
df.style.set_caption(f'{nome_receita} - valores mensais - {dt.year}').hide(axis='index')

#### ICMS

In [ ]:
nome_receita = 'ICMS'
sql = f'''
SELECT
    data_base,
    SUM(receita_realizada) AS arrecadado,
    SUM(meta_mensal) AS meta
FROM pad."RECEITA"

WHERE
    controle = {controle_mes}
    AND entidade LIKE 'pm'
    AND classe_receita NOT LIKE 'intra'
    AND receita_base LIKE '172150%%'
GROUP BY
    data_base
ORDER BY
    data_base ASC
'''
#print(sql)
receita_atual = sqldf(sql)
#receita_atual

sql = f'''
SELECT
    data_base,
    SUM(receita_realizada) AS arrecadado
FROM pad."RECEITA"

WHERE
    controle = {controle_mes_ano_anterior}
    AND entidade LIKE 'pm'
    AND classe_receita NOT LIKE 'intra'
    AND receita_base LIKE '1728011%%'
GROUP BY
    data_base
ORDER BY
    data_base ASC
'''
#print(sql)
receita_anterior = sqldf(sql)
#receita_anterior

receita = receita_atual
receita['anterior'] = receita_anterior['arrecadado']
receita['data_base'] = pd.to_datetime(receita['data_base'], format='%Y-%m-%d')
#receita.dtypes

In [ ]:
acumulado = prepara_df_acumulado(receita.copy())
plot_acumulado(acumulado, nome_receita)

In [ ]:
df = df_table(acumulado.copy(), spec, fmt)
df.style.set_caption(f'{nome_receita} - valores acumulados - {dt.year}').hide(axis='index')
#df

In [ ]:
mensal = prepara_df_mensal(receita.copy())
plot_mensal(mensal, nome_receita)

In [ ]:
df = df_table(mensal.copy(), spec, fmt)
df.style.set_caption(f'{nome_receita} - valores mensais - {dt.year}').hide(axis='index')

### Transferências para a Saúde

Considera as transferências correntes recebidas da União e do Estado para a área da Saúde.

In [ ]:
nome_receita = 'Transferências para a Saúde'
sql = f'''
SELECT
    data_base,
    SUM(receita_realizada) AS arrecadado,
    SUM(meta_mensal) AS meta
FROM pad."RECEITA"

WHERE
    controle = {controle_mes}
    AND entidade LIKE 'pm'
    AND classe_receita NOT LIKE 'intra'
    AND receita_base LIKE '17%%'
    AND fonte_recurso BETWEEN 600 AND 659
GROUP BY
    data_base
ORDER BY
    data_base ASC
'''
#print(sql)
receita_atual = sqldf(sql)
#receita_atual

sql = f'''
SELECT
    data_base,
    SUM(receita_realizada) AS arrecadado
FROM pad."RECEITA"

WHERE
    controle = {controle_mes_ano_anterior}
    AND entidade LIKE 'pm'
    AND classe_receita NOT LIKE 'intra'
    AND receita_base LIKE '171803%%'
    OR receita_base LIKE '171804%%'
    OR receita_base LIKE '1718101%%'
    OR receita_base LIKE '172803%%'
    OR receita_base LIKE '1728101%%'
GROUP BY
    data_base
ORDER BY
    data_base ASC
'''
#print(sql)
receita_anterior = sqldf(sql)
#receita_anterior

receita = receita_atual
receita['anterior'] = receita_anterior['arrecadado']
receita['data_base'] = pd.to_datetime(receita['data_base'], format='%Y-%m-%d')
#receita.dtypes

In [ ]:
acumulado = prepara_df_acumulado(receita.copy())
plot_acumulado(acumulado, nome_receita)

In [ ]:
df = df_table(acumulado.copy(), spec, fmt)
df.style.set_caption(f'{nome_receita} - valores acumulados - {dt.year}').hide(axis='index')
#df

In [ ]:
mensal = prepara_df_mensal(receita.copy())
plot_mensal(mensal, nome_receita)

In [ ]:
df = df_table(mensal.copy(), spec, fmt)
df.style.set_caption(f'{nome_receita} - valores mensais - {dt.year}').hide(axis='index')

### Transferências para a Educação

Considera as transferências correntes recebidas da União e do Estado para a área da Educação (exceto o Fundeb).

In [ ]:
nome_receita = 'Transferências para a Educação'
sql = f'''
SELECT
    data_base,
    SUM(receita_realizada) AS arrecadado,
    SUM(meta_mensal) AS meta
FROM pad."RECEITA"

WHERE
    controle = {controle_mes}
    AND entidade LIKE 'pm'
    AND classe_receita NOT LIKE 'intra'
    AND receita_base LIKE '1714%%'
    OR receita_base LIKE '171751%%'
    OR receita_base LIKE '172451%%'
GROUP BY
    data_base
ORDER BY
    data_base ASC
'''
#print(sql)
receita_atual = sqldf(sql)
#receita_atual

sql = f'''
SELECT
    data_base,
    SUM(receita_realizada) AS arrecadado
FROM pad."RECEITA"

WHERE
    controle = {controle_mes_ano_anterior}
    AND entidade LIKE 'pm'
    AND classe_receita NOT LIKE 'intra'
    AND receita_base LIKE '17%%'
    AND fonte_recurso BETWEEN 550 AND 599
GROUP BY
    data_base
ORDER BY
    data_base ASC
'''
#print(sql)
receita_anterior = sqldf(sql)
#receita_anterior

receita = receita_atual
receita['anterior'] = receita_anterior['arrecadado']
receita['data_base'] = pd.to_datetime(receita['data_base'], format='%Y-%m-%d')
#receita.dtypes

In [ ]:
acumulado = prepara_df_acumulado(receita.copy())
plot_acumulado(acumulado, nome_receita)

In [ ]:
df = df_table(acumulado.copy(), spec, fmt)
df.style.set_caption(f'{nome_receita} - valores acumulados - {dt.year}').hide(axis='index')
#df

In [ ]:
mensal = prepara_df_mensal(receita.copy())
plot_mensal(mensal, nome_receita)

In [ ]:
df = df_table(mensal.copy(), spec, fmt)
df.style.set_caption(f'{nome_receita} - valores mensais - {dt.year}').hide(axis='index')

### Transferências para a Assistência Social

Considera as transferências correntes recebidas da União e do Estado para a área da Assistência Social.

In [ ]:
nome_receita = 'Transferências para a Assistência Social'
sql = f'''
SELECT
    data_base,
    SUM(receita_realizada) AS arrecadado,
    SUM(meta_mensal) AS meta
FROM pad."RECEITA"

WHERE
    controle = {controle_mes}
    AND entidade LIKE 'pm'
    AND classe_receita NOT LIKE 'intra'
    AND receita_base LIKE '17%%'
    AND fonte_recurso BETWEEN 660 AND 669
GROUP BY
    data_base
ORDER BY
    data_base ASC
'''
#print(sql)
receita_atual = sqldf(sql)
#receita_atual

sql = f'''
SELECT
    data_base,
    SUM(receita_realizada) AS arrecadado
FROM pad."RECEITA"

WHERE
    controle = {controle_mes_ano_anterior}
    AND entidade LIKE 'pm'
    AND classe_receita NOT LIKE 'intra'
    AND receita_base LIKE '1718103%%'
    OR receita_base LIKE '171812%%'
    OR receita_base LIKE '172807%%'
    OR receita_base LIKE '1728109107%%'
GROUP BY
    data_base
ORDER BY
    data_base ASC
'''
#print(sql)
receita_anterior = sqldf(sql)
#receita_anterior

receita = receita_atual
receita['anterior'] = receita_anterior['arrecadado']
receita['data_base'] = pd.to_datetime(receita['data_base'], format='%Y-%m-%d')
#receita.dtypes

In [ ]:
acumulado = prepara_df_acumulado(receita.copy())
plot_acumulado(acumulado, nome_receita)

In [ ]:
df = df_table(acumulado.copy(), spec, fmt)
df.style.set_caption(f'{nome_receita} - valores acumulados - {dt.year}').hide(axis='index')
#df

In [ ]:
mensal = prepara_df_mensal(receita.copy())
plot_mensal(mensal, nome_receita)

In [ ]:
df = df_table(mensal.copy(), spec, fmt)
df.style.set_caption(f'{nome_receita} - valores mensais - {dt.year}').hide(axis='index')